<a href="https://colab.research.google.com/github/elnur86/CarPricePrediction/blob/main/electricityPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [283]:
import pandas as pd
import re
from datetime import datetime

In [284]:
df=pd.read_excel("Usage.xlsx", header=None,index_col=None)

In [285]:
df.set_axis(['Usage'], axis='columns', inplace=True)

In [286]:
df

,Usage
0,3 PM Mon 24th-Mar-2014___0.384 kwh
1,5AM 15-Aug-2014___1.201 kwh
2,__8PM Thu 20-Mar-2014____1.523 kwh
3,6PM 23rd-Apr-2014___0.424 kwh
4,_1AM Friday 19th-Dec-2014___0.209 kwh
...,...
8755,_1AM Fri 07th-Nov-2014_0.084 kwh
8756,_6AM 20-May-2014__1.027 kwh
8757,__2 AM Tuesday 8th-Apr-2014___0.052 kwh
8758,9 PM 27th-Jan-2014_1.428 kwh


In [287]:
pattern_for_digits="[0-9]+"
pattern_for_AMPM="AM|PM"
pattern_for_date="[0-9]{1,2}[a-zA-Z]{0,2}-[a-zA-Z]{3,3}-[0-9]{4,4}"
pattern_for_usage="[0-9]{1,2}.[0-9]{1,3}"

In [288]:
df['hour'] = df['Usage'].apply(lambda x: re.findall(pattern_for_digits,x)[0])

In [289]:
df['AMPM']=df['Usage'].apply(lambda x: re.findall(pattern_for_AMPM,x)[0])

In [290]:
df['time']=df['hour']+':00:00 '+df['AMPM']
df['time24']=df['time'].apply(lambda x: pd.to_datetime(x).strftime('%H:%M:%S'))


In [291]:
df['time24']=df['time24'].apply(lambda x: datetime.strptime(x,"%H:%M:%S").time())

In [292]:
df['date']=df['Usage'].apply(lambda x: re.findall(pattern_for_date,x)[0])

In [293]:
df['date']=df['date'].apply(lambda x: x.replace("th",""))

In [294]:
df['date']=df['date'].apply(lambda x: x.replace("st",""))

In [295]:
df['date']=df['date'].apply(lambda x: x.replace("nd",""))

In [296]:
df['date']=df['date'].apply(lambda x: x.replace("rd",""))

In [297]:
df['date']=df['date'].apply(lambda x: datetime.strptime(x,"%d-%b-%Y"))

In [298]:
days=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
df['weekDay']=df['date'].apply(lambda x: days[x.weekday()])

In [299]:
months=["January","February","March","April","May","June","July","August","September","October","November","December"]
df['month']=df['date'].apply(lambda x: months[x.month-1])

In [300]:
df['kwh']=df['Usage'].apply(lambda x: re.findall(pattern_for_usage,x)[-1])


In [301]:
df[["kwh"]] = df[["kwh"]].apply(pd.to_numeric)

In [302]:
df.drop(['Usage','AMPM','hour'], axis=1,inplace=True)

**Question 1** What is your average hourly electricity
usage?

In [303]:
df['kwh'].mean()

0.7817481735159817

**Question 2** What is your average electricity usage
perhour in February?

In [304]:
df.groupby(by='month',as_index=False)['kwh'].mean()

,month,kwh
0,April,0.210355
1,August,0.852304
2,December,0.763689
3,February,0.833735
4,January,1.401425
5,July,1.687300
6,June,1.255096
7,March,0.566562
8,May,0.560987
9,November,0.322806


**Question 3** Which day of the week has the
highestaverage usage?

In [305]:
df.groupby(by='weekDay',as_index=False)['kwh'].mean()

,weekDay,kwh
0,Friday,0.753666
1,Monday,0.725894
2,Saturday,0.890300
3,Sunday,0.898601
4,Thursday,0.746375
5,Tuesday,0.724843
6,Wednesday,0.733699


**Question 4** What is the highest amount of
electricityused in a continuous 4 hour
period?

In [306]:
df['date_time']=df.apply(lambda r : pd.datetime.combine(r['date'],r['time24']),1)

<ipython-input-306-eb5f0c8653ab>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df['date_time']=df.apply(lambda r : pd.datetime.combine(r['date'],r['time24']),1)


In [307]:
df=df.sort_values(by='date_time').reset_index(drop=True)
df

,time,time24,date,weekDay,month,kwh,date_time
0,12:00:00 AM,00:00:00,2014-01-01,Wednesday,January,0.684,2014-01-01 00:00:00
1,1:00:00 AM,01:00:00,2014-01-01,Wednesday,January,0.340,2014-01-01 01:00:00
2,2:00:00 AM,02:00:00,2014-01-01,Wednesday,January,0.332,2014-01-01 02:00:00
3,3:00:00 AM,03:00:00,2014-01-01,Wednesday,January,0.396,2014-01-01 03:00:00
4,4:00:00 AM,04:00:00,2014-01-01,Wednesday,January,0.388,2014-01-01 04:00:00
...,...,...,...,...,...,...,...
8755,7:00:00 PM,19:00:00,2014-12-31,Wednesday,December,1.940,2014-12-31 19:00:00
8756,8:00:00 PM,20:00:00,2014-12-31,Wednesday,December,1.901,2014-12-31 20:00:00
8757,9:00:00 PM,21:00:00,2014-12-31,Wednesday,December,0.766,2014-12-31 21:00:00
8758,10:00:00 PM,22:00:00,2014-12-31,Wednesday,December,0.633,2014-12-31 22:00:00


In [308]:
maxcons4days=0
for i in range(len(df)-4):
  cons4days=df['kwh'][i]+df['kwh'][i+1]+df['kwh'][i+2]+df['kwh'][i+3]
  if cons4days>maxcons4days:
    maxcons4days=cons4days
    index =i
print(maxcons4days,index)

17.237000000000002 4816


In [309]:
df.head(4819).tail(4)

,time,time24,date,weekDay,month,kwh,date_time
4815,4:00:00 PM,16:00:00,2014-07-20,Sunday,July,2.290,2014-07-20 16:00:00
4816,5:00:00 PM,17:00:00,2014-07-20,Sunday,July,2.333,2014-07-20 17:00:00
4817,6:00:00 PM,18:00:00,2014-07-20,Sunday,July,4.709,2014-07-20 18:00:00
4818,7:00:00 PM,19:00:00,2014-07-20,Sunday,July,5.054,2014-07-20 19:00:00


The data cleaning has been completed. From this point onwards tariffs will be compared to find the best fit for us.

Initially checkin No Flex tariff

In [310]:
total_usage=df['kwh'].agg(lambda x : sum(x))

In [311]:
Flat_rate=0.21
No_flex_tariff_total=round(total_usage*Flat_rate,0)
print(No_flex_tariff_total)

1438.0


Monthly Flex tariff

In [312]:
Monthly_flex={
    "January":0.20,
    "February":0.19,
    "March":0.17,
    "April":0.18,
    "May":0.22,
    "June":0.27,
    "July":0.24,
    "August":0.19,
    "September":0.18,
    "October":0.15,
    "November":0.14,
    "December":0.19
}

In [313]:
monthly_groupped = df.groupby(by="month",as_index=False)["kwh"].sum()

In [314]:
monthly_groupped

,month,kwh
0,April,151.666
1,August,635.819
2,December,568.948
3,February,559.436
4,January,1042.660
5,July,1255.351
6,June,899.904
7,March,422.655
8,May,416.813
9,November,232.420


In [315]:
monthly_groupped['total']=monthly_groupped['month'].apply(lambda x: (Monthly_flex[x])*(monthly_groupped[monthly_groupped['month']==x]['kwh'].item()))

**Question 5** Based on your historic electricity usage,
what would your annual cost of
electricitybe under the "Monthly Flex"
contract?

In [316]:
Monthly_flex_tariff_total=round(monthly_groupped['total'].sum(),0)
print(Monthly_flex_tariff_total)

1421.0


Hourly Flex Tariff

In [320]:
Hourly_flex={
    "1:00:00 AM":0.10,
    "2:00:00 AM":0.10,
    "3:00:00 AM":0.10,
    "4:00:00 AM":0.10,
    "5:00:00 AM":0.15,
    "6:00:00 AM":0.20,
    "7:00:00 AM":0.24,
    "8:00:00 AM":0.24,
    "9:00:00 AM":0.26,
    "10:00:00 AM":0.20,
    "11:00:00 AM":0.20,
    "12:00:00 PM":0.26,
    "1:00:00 PM":0.26,
    "2:00:00 PM":0.20,
    "3:00:00 PM":0.24,
    "4:00:00 PM":0.18,
    "5:00:00 PM":0.15,
    "6:00:00 PM":0.30,
    "7:00:00 PM":0.24,
    "8:00:00 PM":0.24,
    "9:00:00 PM":0.12,
    "10:00:00 PM":0.11,
    "11:00:00 PM":0.10,
    "12:00:00 AM":0.10
}

In [321]:
hourly_groupped=df.groupby(by='time',as_index=False)['kwh'].sum()

In [322]:
hourly_groupped

,time,kwh
0,10:00:00 AM,183.174
1,10:00:00 PM,210.731
2,11:00:00 AM,213.951
3,11:00:00 PM,257.327
4,12:00:00 AM,181.638
5,12:00:00 PM,212.546
6,1:00:00 AM,93.966
7,1:00:00 PM,184.716
8,2:00:00 AM,92.192
9,2:00:00 PM,208.166


In [323]:
hourly_groupped['total']=hourly_groupped['time'].apply(lambda x: Hourly_flex[x]*hourly_groupped[hourly_groupped['time']==x]['kwh'].item())

In [324]:
Hourly_flex_tariff_total=round(hourly_groupped['total'].sum(),0)
print(Hourly_flex_tariff_total)

1394.0


Print out of the total amount per each tariff

**Question 6** Based on your historic electricity
usage, which of the three contracts
would pro- duce the lowest annual
cost?

In [325]:
print('No flex tariff total = ',No_flex_tariff_total)
print('Monthly flex tariff total = ',Monthly_flex_tariff_total)
print('Hourly flex tariff total = ',Hourly_flex_tariff_total)


No flex tariff total =  1438.0
Monthly flex tariff total =  1421.0
Hourly flex tariff total =  1394.0
